In [ ]:
# auto-reload modules
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_formats = ['svg']

import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from fastdtw import fastdtw
from scipy.ndimage import gaussian_filter
from scipy.spatial.distance import euclidean
from sklearn.cluster import AffinityPropagation
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from tslearn.clustering import TimeSeriesKMeans, silhouette_score
from tslearn.metrics import cdist_dtw
from tslearn.utils import to_time_series_dataset

sys.path.append("../")

from per_analysis import config
from per_analysis.figures.trajectory import plot_per_trajectory2
from per_analysis.stats import estimate_mode_1d

plt.style.use("../styles/custom.mplstyle")


def preprocess(df):
    df["time"] = df.trial * 15 + df.frame / 20
    angles = [angle for angle in df.columns if "angle" in angle]

    baseline = {
        angle: {
            fly: estimate_mode_1d(df_fly[angle], dx=1)
            for fly, df_fly in df.groupby("fly")
        }
        for angle in angles
    }

    for angle in angles:
        for fly, _ in df.groupby("fly"):
            df.loc[df.fly == fly, angle] -= baseline[angle][fly]

    for fly, df_fly in df.groupby("fly"):
        for angle in angles:
            y = gaussian_filter(df_fly[angle], 2, mode="nearest")
            df.loc[df.fly == fly, angle] = y

    return df


def plot(df_per, xlim, ylim, theta=45, wspace=0, hspace=.5, xlabel="Haustellum angle", ylabel="Rostrum angle", figsize=(4.5, 4.5)):
    fig = plot_per_trajectory2(
        df_per,
        per_angles,
        figsize=figsize,
        spines=False,
        xlim=xlim,
        ylim=ylim,
        xlabel=xlabel,
        ylabel=ylabel,
        quiver=True,
    )

    fig.subplots_adjust(wspace=wspace, hspace=hspace)
    ax = fig.axes[6]
    ax.plot([0, 0], [0, theta], transform=ax.get_yaxis_transform(), c="k", lw=2)
    ax.plot([0, theta], [0, 0], transform=ax.get_xaxis_transform(), c="k", lw=2)
    ax.text(theta / 2, -0.04, f"{theta}°", ha="center", va="top", transform=ax.get_xaxis_transform())
    ax.text(0, theta / 2, f"{theta}°", ha="right", va="center", rotation=90, transform=ax.get_yaxis_transform())

In [ ]:
csv_file = "../data/WT_50%.csv"
df = pd.read_csv(csv_file)
discard = (
    "20190904-1",  # trial 10 only has 36 frames
    "20190904-10",  # inaccurate tracking (e.g., 20190904-10-007 to 20190904-10-09)
    "20190905-5",  # fly fell off in trial 6
)

df = df[~df.fly.isin(discard)]
df = preprocess(df)

df_per = pd.read_csv("../data/WT_50%_per.csv").fillna("")

angles = [angle for angle in df.columns if "angle" in angle]
n_components = 2

# Normalize before passing to PCA
pipeline = make_pipeline(StandardScaler(), PCA(n_components))

# Fit and transform angles
pc_columns = [f"pc{i}" for i in range(n_components)]
Z = pipeline.fit_transform(df[angles])

for i in range(n_components):
    df[f"pc{i}"] = Z[:, i]

per_pcs = []
per_angles = []

for fly, df_fly_per in df_per.groupby("fly"):
    df_fly = df[df.fly == fly]

    for row in df_fly_per.itertuples():
        per_pcs.append(df_fly.iloc[row.start : row.stop][pc_columns].values)
        per_angles.append(df_fly.iloc[row.start : row.stop][angles[:2]].values)

per_pcs = np.array(per_pcs, dtype=object)
per_angles = np.array(per_angles, dtype=object)

n_per = len(df_per)
recompute_matrix = False
distance_matrix_path = Path(csv_file).with_suffix(".npy")

D = None

if distance_matrix_path.exists() and not recompute_matrix:
    D = np.load(str(distance_matrix_path))

    if D.shape != (n_per, n_per):
        D = None

if D is None:
    D = np.zeros((n_per,) * 2)

    for i in range(n_per):
        for j in range(i, n_per):
            D[i, j] = D[j, i] = fastdtw(
                per_pcs[i], per_pcs[j], radius=2, dist=euclidean
            )[0]

    np.save(str(distance_matrix_path), D)

## K-means clustering
- Metric: DTW
- Input: the 3 PCs during each PER
- Test number of clusters K = 2 to 7
- Choose K such that the average silhouette coefficient over the samples is maximized
- For each cluster, choose the trajectory that is closest to the center of the cluster as the exemplar of that cluster

In [ ]:
kmeans_dict = {}
k_range = np.arange(2, 8)

for odor, df_odor_per in df_per[df_per.odor != ""].groupby("odor"):
    X = to_time_series_dataset(per_pcs[df_odor_per.index])
    kmeans_dict[odor] = [
        TimeSeriesKMeans(n_clusters=k, metric="dtw", random_state=0).fit(X)
        for k in k_range
    ]

min_cluster_size = 5
km_exemplar_idx = []

for odor, df_odor_per in df_per[df_per.odor != ""].groupby("odor"):
    X = to_time_series_dataset(per_pcs[df_odor_per.index])
    kmeans_list = kmeans_dict[odor]
    cdist = cdist_dtw(X)
    argmax = np.argmax(
        [
            silhouette_score(cdist, km.labels_, metric="precomputed")
            for km in kmeans_list
        ]
    )
    km = kmeans_list[argmax]
    dist_to_center = cdist_dtw(
        X, km.cluster_centers_[np.bincount(km.labels_) >= min_cluster_size]
    )
    km_exemplar_idx.extend(
        df_odor_per.index.values[dist_to_center.argmin(0)]
    )

xlim, ylim = np.array(
    [np.concatenate(per_angles).min(0), np.concatenate(per_angles).max(0)]
).T

plot(df_per.loc[km_exemplar_idx], xlim=xlim, ylim=ylim)

## K-means clustering
- Same as above but choose K by the elbow method

In [ ]:
odors_k = {
    "2-PT": 4,
    "Air": 4,
    "BNZ": 4,
    "EBR": 4,
    "IPA": 4,
    "MCH": 5,
    "MO": 5,
    "OCT": 5,
    "Water": 5,
}

fig, axes = plt.subplots(3, 3, figsize=(4, 3), sharex="all")

for i, odor in enumerate(config.ODOR_COL_ORDER):
    ax = axes.ravel()[i]
    kmeans_list = kmeans_dict[odor]
    ax.plot(k_range, [km.inertia_ for km in kmeans_list], c="k", lw=1)
    ax.set_xticks(k_range)
    ax.set_yticks([])
    ax.set_title(odor)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.axvline(odors_k[odor], ls="--", color="b", lw=1)

axes[-1, 1].set_xlabel("K")
axes[1, 0].set_ylabel("Inertia")
plt.tight_layout()

min_cluster_size = 5
km_exemplar_idx = []

for odor, df_odor_per in df_per[df_per.odor != ""].groupby("odor"):
    X = to_time_series_dataset(per_pcs[df_odor_per.index])
    km = kmeans_dict[odor][np.argwhere(k_range == odors_k[odor]).item()]
    dist_to_center = cdist_dtw(
        X, km.cluster_centers_[np.bincount(km.labels_) >= min_cluster_size]
    )
    km_exemplar_idx.extend(
        df_odor_per.index.values[dist_to_center.argmin(0)]
    )

plot(df_per.loc[km_exemplar_idx], xlim=xlim, ylim=ylim)

## [Affinity propagation clustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html)
- Perform clustering and find exemplars of clusters
- See https://scikit-learn.org/stable/modules/clustering.html#affinity-propagation

In [ ]:
ap_exemplar_idx = []

for odor, df_odor_per in df_per[df_per.odor != ""].groupby("odor"):
    D_odor = D[df_odor_per.index][:, df_odor_per.index]
    ap = AffinityPropagation(affinity="precomputed", random_state=0).fit(
        -D_odor
    )
    idx = df_odor_per.index.values[
        ap.cluster_centers_indices_[np.bincount(ap.labels_) >= 5]
    ]
    ap_exemplar_idx.extend(idx)

plot(df_per.loc[ap_exemplar_idx], xlim=xlim, ylim=ylim)